### Краткое описание файла 04_ml_data_prep

На данном этапе  подготовим данных для обучения и дообучения современных трансформерных моделей (Roberta-base, sentence-transformers MiniLM) по задачам извлечения навыков из текста вакансии и классификации грейда.

- Сформируем размеченные датасеты для задач NER и классификации.
- Проверен баланс классов, очистим пропуски, проведем базовую EDA.



In [3]:
import pandas as pd
import numpy as np

In [5]:
#грузим основной датасет с доп признаками
df = pd.read_csv('data/raw/feature_vacancies.csv')
df.head()

,id,title,published_at,description,salary_from,salary_to,salary_currency,experience_hh,area_id,skills_raw,salary_rub,desc_len,desc_words,title_len,num_skills,exp_junior,exp_middle,exp_senior,exp_lead
0,120682290,Водитель с личным автомобилем,2025-05-19T09:30:49+0300,<p>Вакансия &quot;Водитель с личным автомобиле...,NaN,90000.0,RUR,Более 6 лет,3,Вождение автомобилей представительского класса...,90000.0,1397,155,29,5,0,0,0,1
1,120761341,Middle/Senior Frontend разработчик,2025-05-20T13:03:11+0300,<p><strong>О компании и команде</strong></p> <...,NaN,3000.0,USD,От 3 до 6 лет,2,"JavaScript, React, CSS, Node.js, TypeScript, W...",270000.0,2242,249,34,6,0,0,1,0
2,120615179,Упаковщик,2025-05-19T09:06:01+0300,"<p><strong><em>Крупная, стабильно развивающаяс...",45000.0,48000.0,RUR,Нет опыта,66,NaN,46500.0,1122,114,9,0,1,0,0,0
3,120693457,Управляющий рестораном семейного концепта,2025-05-19T11:16:42+0300,<p><strong>Приглашаем Управляющего семейным ре...,NaN,NaN,NaN,От 1 года до 3 лет,78,NaN,NaN,3681,362,41,0,0,1,0,0
4,120219268,Медицинский работник (Российские студенческие ...,2025-05-05T11:37:27+0300,<p>​​​​​​Направление студенческих медицинских ...,20000.0,35000.0,RUR,Нет опыта,1,"Ответственность, Стрессоустойчивость",27500.0,1183,132,53,2,1,0,0,0


##Подготовим тренировочные выборки для ML
a) Skill-NER (для дообучения Roberta-base)
Соберем пары: текст описания вакансии (description) и список навыков (skills_raw).

Удалим строки без навыков.

Приведем навыки к нижнему регистру, почистим пробелы.

In [6]:
ner_df = df[['description', 'skills_raw']].dropna()
ner_df = ner_df[ner_df['skills_raw'].str.strip() != '']
# Приводим к списку
ner_df['skills_list'] = ner_df['skills_raw'].apply(lambda x: [s.strip().lower() for s in x.split(',') if s.strip()])
ner_df = ner_df[ner_df['skills_list'].apply(len) > 0]
print(ner_df.head())

                                         description  \
0  <p>Вакансия &quot;Водитель с личным автомобиле...   
1  <p><strong>О компании и команде</strong></p> <...   
4  <p>​​​​​​Направление студенческих медицинских ...   
5  <div> <p><strong>edna – аккредитованная IT-ком...   
6  <p><strong>CAD Exchanger</strong> – IT-компани...   

                                          skills_raw  \
0  Вождение автомобилей представительского класса...   
1  JavaScript, React, CSS, Node.js, TypeScript, W...   
4               Ответственность, Стрессоустойчивость   
5       React, HTML, CSS, Redux, TypeScript, Webpack   
6  Git, JavaScript, 3D Моделирование, ООП, WebGL,...   

                                         skills_list  
0  [вождение автомобилей представительского класс...  
1  [javascript, react, css, node.js, typescript, ...  
4             [ответственность, стрессоустойчивость]  
5     [react, html, css, redux, typescript, webpack]  
6  [git, javascript, 3d моделирование, ооп, webgl..

In [8]:
#Сохраним файл для дообучения NER
ner_df[['description', 'skills_list']].to_json('data/raw/ner_train.json', orient='records', lines=True, force_ascii=False)

б) **Грейд-классификатор** (для Roberta/MiniLM или TF-IDF+LogReg)
Маппируем поле experience_hh на классы (junior/middle/senior/lead).

Подготовим X: описание/название, У: класс.

In [10]:
grade_map = {
    'Нет опыта': 0,          #junior
    'От 1 года до 3 лет': 1, #middle
    'От 3 до 6 лет': 2,      #senior
    'Более 6 лет': 3         #lead
}
df['grade_class'] = df['experience_hh'].map(grade_map)
grade_df = df[['description', 'title', 'grade_class']].dropna()
grade_df = grade_df[grade_df['grade_class'].notnull()]
grade_df.to_csv('data/raw/grade_train.csv', index=False)

c) Для **sentence-transformers**/paraphrase-multilingual-MiniLM
Сохраним датасет с парами (“текст вакансии”, “навык”), например для задачи мультилэйбл классификации или эмбеддинга.

In [11]:
mini_pairs = []
for _, row in ner_df.iterrows():
    for skill in row['skills_list']:
        mini_pairs.append({'text': row['description'], 'skill': skill})
mini_df = pd.DataFrame(mini_pairs)
mini_df.to_csv('data/raw/mini_train.csv', index=False)

In [12]:
#Проверка баланса классов и базовой аналитики по датасетам
print("Skill-NER тренировочных примеров:", len(ner_df))
print("Grade-классы (распределение):\n", grade_df['grade_class'].value_counts())

Skill-NER тренировочных примеров: 1709
Grade-классы (распределение):
 grade_class
1    1315
0     774
2     688
3     218
Name: count, dtype: int64


## Выводы по подготовке данных для дообучения моделей

- Проведена подготовка обучающих выборок для двух ML-задач: извлечение навыков (Skill-NER) и классификация грейда вакансии.
- Для Skill-NER собрано 1709 уникальных примеров пар (“текст вакансии” — “навыки”), что позволит дообучить трансформерную модель (Roberta-base) на задаче последовательного извлечения навыков.
- Для классификатора грейда (junior/middle/senior/lead) сформировано 2996 примеров с разметкой на основе поля `experience_hh`.
- Для дообучения sentence-transformers MiniLM подготовлен датасет пар (“текст вакансии”, “навык”) для мультилэйбл классификации и поиска по эмбеддингам.
- Данные проверены на баланс классов и полноту, сохранены в формате для быстрого старта обучения в Colab или на локальной машине.
- Этап подготовки данных позволяет перейти к полноценному ML: дообучить глубокие модели и интегрировать их в сервис Telegram-бота.
